In [ ]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
 
default = {'scenario': 'port-moresby', 'training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [ ]:
import os
import time
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)
from syspy.spatial import spatial, utils
io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

In [ ]:
scenario = argv['scenario']

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario  + '/inputs/'
    output_folder = training_folder + '/scenarios/' + scenario + '/outputs/'
    model_folder = training_folder + '/scenarios/' + scenario  + '/model/'
else:
    scenario_folder = input_folder
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)

In [ ]:
if 'params' in argv.keys():
    scenario = argv['params']['general']['scenario'] #'params':{'general':{'scenario':'ref2023'}}
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
else:
    var = excel.read_var(file=input_folder+'parameters.xlsx', scenario=scenario, return_ancestry=False)

In [ ]:
def round_up(n, decimals=0):
    multiplier = 10**decimals
    return math.ceil(n * multiplier) / multiplier

In [ ]:
sm = stepmodel.read_zippedpickles(model_folder +'logit_assignment')
sm.segments = ['car_owner', 'pt_captive'] 

In [ ]:
sm = sm.change_epsg(4326,'degree')

In [ ]:
comp_provided = os.path.isfile((training_folder +'/scenarios/{scen}_projet/model/logit_assignment'.format(scen = scenario)))
if not comp_provided:
    end_of_notebook

In [ ]:
new = stepmodel.read_zippedpickles(training_folder +'/scenarios/{scen}_projet/model/logit_assignment'.format(scen = scenario))

In [ ]:
new.links.loc[new.links['agency_id'] == 'QUENEDI'].groupby('route_id')['boardings'].sum()

In [ ]:
comp_utility =(new.utilities.set_index(['origin','destination'])['pt']-sm.utilities.set_index(['origin','destination'])['pt']).replace(np.inf, np.nan).replace(-np.inf, np.nan)

In [ ]:
comp_utility.describe()

In [ ]:
new.utilities

In [ ]:
comp_utility.hist()

In [ ]:
comp_boardings_agency = pd.concat([sm.links.groupby(['agency_id'])['boardings'].sum(), new.links.groupby(['agency_id'])['boardings'].sum()],axis=1, keys = ['ref', 'new'])
comp_boardings_agency

In [ ]:
comp_boardings_mode = pd.concat([sm.links.groupby(['route_type'])['boardings'].sum(), new.links.groupby(['route_type'])['boardings'].sum()],axis=1, keys = ['boardings_ref', 'boardings_new'])
comp_boardings_mode

In [ ]:
comp_boardings_trip = pd.concat([sm.links.groupby('trip_id')['boardings'].sum(), new.links.groupby('trip_id')['boardings'].sum()],axis=1, keys = ['ref', 'new'])
comp_boardings_trip

In [ ]:
comp_boardings_route = pd.concat([sm.links.groupby('route_id')['boardings'].sum(), new.links.groupby('route_id')['boardings'].sum()],axis=1, keys = ['ref', 'new'])
comp_boardings_route

In [ ]:
def plot_time_var(to_plot, 
                    voy_h_sm=0,
                    voy_h_new=0, 
                    title=None):

    systra_color = ["#D22328", "#559BB4", "#91A564", "#DC9100", "#8C4B7D", "#A08C69","#647D6E", "#5A7382", "#64411E", "#A00037", "#643C5A"]
    to_plot = to_plot.iloc[1:18]
    to_plot.index = ['{} - {} min'.format(i*5, (i+1)*5) for i in to_plot.index]
    to_plot.columns = ['Projet', 'Référence']

    to_plot.plot(kind='bar', figsize=[20, 10], fontsize=15, color=systra_color, zorder=3)
    plt.xticks(rotation=70)
    plt.grid(axis='y',zorder=0)
    plt.title(title, loc='left', fontsize=18, fontweight="bold")
    plt.legend(['Projet: ({} voy.h)'.format(int(np.round(voy_h_sm, -2))), 'Référence ({} voy.h)'.format(int(np.round(voy_h_new, -2)))], loc='upper right', fontsize=14)

In [ ]:
sm.los['time_class'] = sm.los['time'].apply(lambda d: int(divmod(d,300)[0]))
new.los['time_class'] = new.los['time'].apply(lambda d: int(divmod(d,300)[0]))
voy_h_sm = (sm.los['volume']*sm.los['time']/3600).sum()
voy_h_new = (new.los['volume']*new.los['time']/3600).sum()
time_class_sm = sm.los.groupby('time_class')['volume'].sum()
time_class_new = new.los.groupby('time_class')['volume'].sum()  
to_plot = pd.merge(time_class_sm, time_class_new, right_index=True, left_index=True)
plot_time_var(to_plot,
                    voy_h_sm=voy_h_sm,
                    voy_h_new=voy_h_new)

In [ ]:
links = new.links.copy()
links.groupby("trip_id").agg({'headway':"mean", 'route_type' : 'first',"length":"sum"})
links["vehicules"] = 10740/links["headway"]
vkm_new = links.groupby("route_type").agg({'vehicules':"sum", "length":"sum"})
links['vkm_new'] = (links["vehicules"]/1000) * links["length"]
vkm_new = links.groupby('route_type').agg({'vkm_new':"sum"})

links = sm.links.copy()
links.groupby("trip_id").agg({'headway':"mean", 'route_type' : 'first',"length":"sum"})
links["vehicules"] = 10740/links["headway"]
vkm_ref = links.groupby("route_type").agg({'vehicules':"sum", "length":"sum"})
links['vkm_ref'] = (links["vehicules"]/1000) * links["length"]
vkm_ref = links.groupby('route_type').agg({'vkm_ref':"sum"})

vkm = pd.concat([vkm_ref, vkm_new], axis=1)
vkm

In [ ]:
results = pd.concat([vkm, comp_boardings_mode], axis=1)
results

In [ ]:
end_of_notebook